In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from functools import partial

In [ ]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

In [ ]:
# Download Dataset from https://www.dropbox.com/scl/fi/v7c1c8a3cnncuv1fo28es/Wages.xlsx?rlkey=vli12nwph687hvn9jskgf73a1&st=s862pfm6&dl=1
# and add it to colab

In [ ]:
wagesDf = pd.read_excel("./Wages.xlsx")
# wagesDf = pd.read_excel("https://www.dropbox.com/scl/fi/v7c1c8a3cnncuv1fo28es/Wages.xlsx?rlkey=vli12nwph687hvn9jskgf73a1&st=s862pfm6&dl=1")

In [ ]:
wagesDf

In [ ]:
wagesDf.size

In [ ]:
wagesDf.describe()

In [ ]:
wagesDf.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plotting
fig1 = plt.figure(
  figsize=(8, 8)
)

In [ ]:
def wageModel3Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    if isinstance(dataForTransfer, pd.DataFrame):
        df = dataForTransfer.copy()
    else:
        df = pd.DataFrame(dataForTransfer)
    df['agePower2'] = df.apply(lambda row: row['Age'] * row['Age'], axis=1)
    dfTransformer = sm.add_constant(df[['Age', 'agePower2']],has_constant='add')
    return dfTransformer

In [ ]:
wagesDf['agePower2'] = wageModel3Transformer(wagesDf)['agePower2']
wagesDf

In [ ]:
wagesDf = wagesDf.sort_values(by="Age")

In [ ]:
wageModel3 = sm.OLS(
  wagesDf["Wage"],
  wageModel3Transformer(wagesDf)
)
wageModel3Fit = wageModel3.fit()
print(wageModel3Fit.summary())

In [ ]:
wageModel3Fit.params

In [ ]:
predictedWage3 = wageModel3Fit.predict(wageModel3Transformer(wagesDf))
wagesDf['predictedWage3'] = predictedWage3
wagesDf

In [ ]:
# Plotting
plt.figure(
  figsize=(8, 8)
)

plt.scatter(
  wagesDf["Age"],
  wagesDf["Wage"],
  color='blue',
  alpha=0.9,
  label='Data Points - scatter',
)

plt.plot(
  wagesDf["Age"],
  wagesDf["predictedWage3"],
  color='green',
  label='OLS Regression - predictedWage3'
)
plt.title('Age. Wage with OLS Regression')
plt.xlabel('Age')
plt.ylabel('Wage K')
plt.legend()
plt.grid(True)



plt.show()

In [ ]:
wagesDf

In [ ]:
# Extract coefficients
coefficients = wageModel3Fit.params
intercept = coefficients['const']
slope_age = coefficients['Age']
slope_age2 = coefficients['agePower2']

In [ ]:


# Solve the quadratic equation
a = slope_age2
b = slope_age
c = intercept

# Calculate the discriminant
discriminant = b**2 - 4*a*c

if discriminant >= 0:
    root1 = (-b + np.sqrt(discriminant)) / (2*a)
    root2 = (-b - np.sqrt(discriminant)) / (2*a)
    roots = [root1, root2]
else:
    roots = []

# Plotting the quadratic curve and the roots
age_values = np.linspace(-5, 100, 400)
wage_predictions = intercept + slope_age * age_values + slope_age2 * (age_values**2)

plt.figure(figsize=(10, 6))
plt.plot(age_values, wage_predictions, label='Quadratic Fit', color='blue')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')

if roots:
    for root in roots:
        plt.plot(root, 0, 'ro')  # Plot the roots on the curve
        plt.annotate(f'Root: {root:.2f}', (root, 0), textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel('Age')
plt.ylabel('Predicted Wage')
plt.title('Quadratic Fit of Wage vs Age with Roots')
plt.legend()
plt.grid(True)
plt.show()

print(f"The roots of the quadratic equation are: {roots}")


In [ ]:


# Solve the quadratic equation
a = slope_age2
b = slope_age
c = intercept

# Calculate the discriminant
discriminant = b**2 - 4*a*c

if discriminant >= 0:
    root1 = (-b + np.sqrt(discriminant)) / (2*a)
    root2 = (-b - np.sqrt(discriminant)) / (2*a)
    roots = [root1, root2]
else:
    roots = []

# Plotting the quadratic curve and the roots
age_values = np.linspace(-5, 100, 400)
wage_predictions = intercept + slope_age * age_values + slope_age2 * (age_values**2)

plt.figure(figsize=(10, 6))
plt.plot(age_values, wage_predictions, label='Quadratic Fit', color='blue')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')

if roots:
    for root in roots:
        plt.plot(root, 0, 'ro')  # Plot the roots on the curve
        plt.annotate(f'Root: {root:.2f}', (root, 0), textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel('Age')
plt.ylabel('Predicted Wage')
plt.title('Quadratic Fit of Wage vs Age with Roots')
plt.legend()

# Find the vertex (maximum point for a downward parabola)
vertex_age = -b / (2 * a)
vertex_wage = intercept + slope_age * vertex_age + slope_age2 * (vertex_age**2)
# Plot the maximum point (vertex)
plt.plot(vertex_age, vertex_wage, 'go')  # Green dot for the maximum point
plt.annotate(f'Max: ({vertex_age:.2f}, {vertex_wage:.2f})', (vertex_age, vertex_wage), textcoords="offset points", xytext=(0,10), ha='center')



plt.grid(True)
plt.show()

print(f"The roots of the quadratic equation are: {roots}")

In [ ]:
# SST and SSR 
# more information https://365datascience.com/tutorials/statistics-tutorials/sum-squares/

# Extract observed and predicted values
observedValues = wagesDf["Wage"]
predictedValues = wageModel3Fit.predict(wageModel3Transformer(wagesDf))

# Calculate the mean of observed values
mean_observed = np.mean(observedValues)

# Calculate SST
sst = np.sum((observedValues - mean_observed) ** 2)

# Calculate SSR
ssr = np.sum((observedValues - predictedValues) ** 2)

errors = np.sum(( predictedValues - mean_observed) ** 2)

# Calculate R^2
r_squared = 1 - (ssr / sst)
r_squared2 = errors / sst

print(f"SST: {sst}")
print(f"SSR: {ssr}")
print(f"R^2: {r_squared}")
print(f"R^2_2: {r_squared2}")

In [ ]:
wagesDf.shape

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train and test sets
trainSet, testSet = train_test_split(wagesDf, test_size=0.15, random_state=800)
# trainSet, testSet = train_test_split(wagesDf, test_size=0.15)

trainSet.head(), trainSet.shape

In [ ]:
wagesDf.shape, trainSet.shape, testSet.shape

In [ ]:
trainModel = sm.OLS(
  trainSet["Wage"],
  wageModel3Transformer(trainSet)
)
trainModelFit = trainModel.fit()
print(trainModelFit.summary())

In [ ]:
print(wageModel3Fit.summary())

In [ ]:
predictedTest = trainModelFit.predict(wageModel3Transformer(testSet))
testSet['predictedTest'] = predictedTest
testSet

In [ ]:
testSet['error']  = testSet['Wage'] - testSet['predictedTest']
testSet

In [ ]:
# Plot errors
plt.scatter(range(len(testSet['error'])), testSet['error'])
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('Index')
plt.ylabel('Error')
plt.title('Error Plot')
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(testSet['Wage'], testSet['predictedTest']))
print("RMSE:", rmse)

In [ ]:
# RMSE with respect to mean of train set wage
meanTrainWage = np.mean(testSet['Wage'])
rmseMean = np.sqrt(mean_squared_error(testSet['Wage'], [meanTrainWage] * len(testSet['Wage'])))
print("RMSE with respect to mean of train set wage:", rmseMean)

In [ ]:
def wageModel5Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    if isinstance(dataForTransfer, pd.DataFrame):
        df = dataForTransfer.copy()
    else:
        df = pd.DataFrame(dataForTransfer)
    df['agePower2'] = df.apply(lambda row: row['Age'] * row['Age'], axis=1)
    dfTransformer = sm.add_constant(df[['Educ','Age', 'agePower2']],has_constant='add')
    return dfTransformer

In [ ]:
# Lets include Education
trainModel2 = sm.OLS(
  trainSet["Wage"],
  wageModel5Transformer(trainSet)
)
trainModel2Fit = trainModel2.fit()
print(trainModel2Fit.summary())

In [ ]:
predictedTest2 = trainModel2Fit.predict(wageModel5Transformer(testSet))
testSet['predictedTest2'] = predictedTest2
testSet['error2']  = testSet['Wage'] - testSet['predictedTest2']
testSet


In [ ]:
# Plot errors
plt.scatter(range(len(testSet['error'])), testSet['error'])
plt.scatter(range(len(testSet['error2'])), testSet['error2'])
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('Index')
plt.ylabel('Error')
plt.title('Error Plot')
plt.show()


In [ ]:
rmse2 = np.sqrt(mean_squared_error(testSet['Wage'], testSet['predictedTest2']))
print("RMSE:", rmse)
print("RMSE2:", rmse2)

In [ ]:
print("RMSE with respect to mean of train set wage:", rmseMean)

In [ ]:
wage1SampleData = {
    "Educ": [12],
    "Age": [76]
}

In [ ]:
def generalRegressionPredictor(self, transformedData):
    return self.predict(transformedData)

In [ ]:
trainModel2Fit.transformer = wageModel5Transformer
trainModel2Fit.mainPredictor = partial(generalRegressionPredictor, trainModel2Fit)
registerAJrjModel(
    trainModel2Fit,
    {
        "modelName":f"taoyu_ma__to_predictModelBest",
        "version":"1.0.1",
        "params": trainModel2Fit.params.to_dict(),
        "score": -1 * rmse2,
        "modelLibraray": 'sm.OLS',
        "libraryMetadata": {
            "pvalues": trainModel2Fit.pvalues.to_dict(),
            "r_squared": float(trainModel2Fit.rsquared),
            "adj_r_squared": float(trainModel2Fit.rsquared_adj)
        },
    
        "sampleData": {
            "dataForTransfer": wage1SampleData
        }
    }
)

# K-Fold Cross validation

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=55)


In [ ]:
check = kf.split(wagesDf)
check
experiment = 1
# Loop through each fold
# Initialize variables to store results
bestModel = None
bestMse = 100000
mseScores = []
meanMseScores = []

for train_index, val_index in check:
    # Split the data
    trainSet, valSet = wagesDf.iloc[train_index], wagesDf.iloc[val_index]

    # Fit the model
    trainModel = sm.OLS(trainSet["Wage"], wageModel5Transformer(trainSet))
    trainModelFit = trainModel.fit()

    # Predict on the validation set
    val_predictions = trainModelFit.predict(wageModel5Transformer(valSet))

    # Calculate the mean squared error
    mse = mean_squared_error(valSet["Wage"], val_predictions)
    meanMse = mean_squared_error(valSet["Wage"], [np.mean(valSet["Wage"])] * len(valSet["Wage"]))
    mseScores.append(mse)
    meanMseScores.append(meanMse)
    if mse < bestMse:
        bestMse = mse
        bestModel = trainModelFit

    # Print summary for each fold (optional)
    print(f'expr={experiment}')
    experiment = experiment +1
    print(trainModelFit.summary())

In [ ]:
mseScores,meanMseScores

In [ ]:
rmseScores = np.sqrt(mseScores)
rmeanMseScores = np.sqrt(meanMseScores)
rmseScores,rmeanMseScores

In [ ]:
# Calculate average MSE
averageMse = sum(mseScores) / len(mseScores)
averageMseMean = sum(meanMseScores) / len(mseScores)
print(f"Average MSE across all folds: {averageMse}")
print(f"Average MSE Mean across all folds: {averageMseMean}")
print(f"Average RMSE across all folds: {pow(averageMse, 0.5)}")
print(f"Average RMSE Mean across all folds: {pow(averageMseMean, 0.5)}")

In [ ]:
bestModel.transformer = wageModel5Transformer
bestModel.mainPredictor = partial(generalRegressionPredictor, bestModel)
registerAJrjModel(
    bestModel,
    {
        "modelName":f"taoyu_ma__to_predictModelBestCrossValidation",
        "version":"1.0.1",
        "params": bestModel.params.to_dict(),
        "score": -1 * bestMse,
        "modelLibraray": 'sm.OLS',
        "libraryMetadata": {
            "pvalues": bestModel.pvalues.to_dict(),
            "r_squared": float(bestModel.rsquared),
            "adj_r_squared": float(bestModel.rsquared_adj)
        },
    
        "sampleData": {
            "dataForTransfer": wage1SampleData
        }
    }
)